In [ ]:
!pip install opencv-python mediapipe scikit-learn matplotli

In [52]:
!pip install tensorflow

  Obtaining dependency information for tensorflow from https://files.pythonhosted.org/packages/93/21/9b035a4f823d6aee2917c75415be9a95861ff3d73a0a65e48edbf210cec1/tensorflow-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for tensorflow-intel==2.15.0 from https://files.pythonhosted.org/packages/4c/48/1a5a15517f18eaa4ff8d598b1c000300b20c1bb0e624539d702117a0c369/tensorflow_intel-2.15.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for gast!=0.5.0,!=0.5.1,!=0.5.2,>=0.2.1 from https://files.pythonhosted.org/packages/fa/39/5aae571e5a5f4de9c3445dae08a530498e5c53b0e74410eeeb0991c79047/gast-0.5.4-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/57.5 kB ? eta -:--:--
     --------------------- ------------------ 30.7/57.5 kB 1.3 MB/s eta 0:00:01
     ---------------------------------------- 57.5/57.5 kB 1.0 MB/s eta 0:00:00
  Obtaining dependency information for libclang>=13.0.0 from https://files.pythonhosted.org/pa

## 1. Import Dependencies

In [19]:
%matplotlib inline
import cv2
import time
import os

import numpy as np
import mediapipe as mp

from matplotlib import pyplot as plt

In [20]:
mp_holistic = mp.solutions.holistic # Holistic model
mp_drawing = mp.solutions.drawing_utils # Drawing utilities

## 2. Keypoint Detection Function using MP

In [21]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [22]:
def draw_styled_landmarks(image, results):
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(80,22,10), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(80,44,121), thickness=2, circle_radius=2)
                             ) 
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             )

## 3. Folder Creation

In [44]:
#Making the directories to store the files
current = os.getcwd()

actions = np.array(["Hello", "Sorry", "Why"])

path = os.path.join(current, "Videos")

video_count = 20

In [33]:
for action in actions: 
    for count in range(1, video_count+1):
        try:
            file = str(action) + "_" + str(count)
            os.makedirs(os.path.join(path.replace("Videos", "Words"), action, file))
        except:
            pass

## 4. Keypoint Value Extraction Function

In [34]:
def extract_keypoints(results):    

    if results.pose_landmarks:
        pose = np.array([[po.x, po.y, po.z] for po in results.pose_landmarks.landmark]).flatten()
    else:
        pose = np.zeros(33*3)

    if results.left_hand_landmarks:
        left = np.array([[lh.x, lh.y, lh.z] for lh in results.left_hand_landmarks.landmark]).flatten()
    else:
        left = np.zeros(21*3)

    if results.right_hand_landmarks:
        right = np.array([[rh.x, rh.y, rh.z] for rh in results.right_hand_landmarks.landmark]).flatten()
    else:
        right = np.zeros(21*3)
    
    return np.concatenate([pose, left, right])

In [38]:
def extract_coordinates(filepath):
    cap = cv2.VideoCapture(filepath)
    frame_num = 1
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while(frame_num < 31):
            ret, frame = cap.read()
            
            if not ret:
                break
            
            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)
            keypoints = extract_keypoints(results)
            
            npy_path = os.path.join(filepath[:-4].replace("Videos", "Words"), str(frame_num))
            np.save(npy_path, keypoints)
            
            frame_num += 1
    cap.release()

## 5. Data collection

In [37]:
#gets the full file paths and appends it to the list
lt = []
for action in actions:
    for files in os.listdir(os.path.join(path, action)):
        lt.append(os.path.join(path, action, files))

['C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_1.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_10.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_11.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_12.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_13.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_14.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_15.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_16.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_17.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_18.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_19.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_2.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_20.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop\\Videos\\Sorry\\Sorry_3.mp4',
 'C:\\Users\\keith\\OneDrive\\Desktop

In [39]:
for i in lt:
    extract_coordinates(i)

# 6. Data Preprocessing and Labeling

In [53]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [45]:
label_map = {label:num for num, label in enumerate(actions)}
label_map

{'Hello': 0, 'Sorry': 1, 'Why': 2}

In [46]:
sequences, labels = [], []
for action in actions:
    for num in range(1, video_count + 1):
        window = []
        for frame_num in range(1, 31):
            file = str(action) + "_" + str(count)
            res = np.load(os.path.join(path.replace("Videos", "Words"), action, file, "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [47]:
np.array(sequences).shape

(60, 30, 225)

In [55]:
x = np.array(sequences)
y = to_categorical(labels).astype(int)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.1)

In [58]:
x.shape

(60, 30, 225)

## Building and Training LSTM Model

In [65]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [68]:
log_dir = os.path.join(current, 'Logs')
tb_callback = TensorBoard(log_dir = log_dir)

In [60]:
model = Sequential()
model.add(LSTM(64, return_sequences = True, activation = "relu", input_shape = (30, 225)))
model.add(LSTM(128, return_sequences = True, activation = "relu"))
model.add(LSTM(64, return_sequences = False, activation = "relu"))
model.add(Dense(64, activation = "relu"))
model.add(Dense(32, activation = "relu"))
model.add(Dense(actions.shape[0], activation = "softmax"))

In [66]:
opt = keras.optimizers.Adam(learning_rate=0.01)
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['categorical_accuracy'])

In [70]:
model.fit(x_train, y_train, epochs = 2000, callbacks = [tb_callback])

Epoch 1/2000
2/2 [==============================] - 1s 41ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 2/2000
2/2 [==============================] - 0s 40ms/step - loss: 1.0978 - categorical_accuracy: 0.3519
Epoch 3/2000
2/2 [==============================] - 0s 43ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 4/2000
2/2 [==============================] - 0s 43ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 5/2000
2/2 [==============================] - 0s 45ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 6/2000
2/2 [==============================] - 0s 42ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 7/2000
2/2 [==============================] - 0s 43ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 8/2000
2/2 [==============================] - 0s 40ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 9/2000
2/2 [==============================] - 0s 45ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
E

2/2 [==============================] - 0s 43ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 75/2000
2/2 [==============================] - 0s 46ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 76/2000
2/2 [==============================] - 0s 46ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 77/2000
2/2 [==============================] - 0s 44ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 78/2000
2/2 [==============================] - 0s 40ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 79/2000
2/2 [==============================] - 0s 43ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 80/2000
2/2 [==============================] - 0s 44ms/step - loss: 1.0979 - categorical_accuracy: 0.3519
Epoch 81/2000
2/2 [==============================] - 0s 46ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 82/2000
2/2 [==============================] - 0s 45ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 

2/2 [==============================] - 0s 44ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 147/2000
2/2 [==============================] - 0s 41ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 148/2000
2/2 [==============================] - 0s 41ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 149/2000
2/2 [==============================] - 0s 43ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 150/2000
2/2 [==============================] - 0s 41ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 151/2000
2/2 [==============================] - 0s 42ms/step - loss: 1.0978 - categorical_accuracy: 0.3519
Epoch 152/2000
2/2 [==============================] - 0s 39ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 153/2000
2/2 [==============================] - 0s 45ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 154/2000
2/2 [==============================] - 0s 46ms/step - loss: 1.0977 - categorical_accuracy: 0.351

2/2 [==============================] - 0s 43ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 219/2000
2/2 [==============================] - 0s 41ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 220/2000
2/2 [==============================] - 0s 44ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 221/2000
2/2 [==============================] - 0s 50ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 222/2000
2/2 [==============================] - 0s 54ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 223/2000
2/2 [==============================] - 0s 53ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 224/2000
2/2 [==============================] - 0s 51ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 225/2000
2/2 [==============================] - 0s 46ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 226/2000
2/2 [==============================] - 0s 41ms/step - loss: 1.0977 - categorical_accuracy: 0.351

2/2 [==============================] - 0s 46ms/step - loss: 1.0978 - categorical_accuracy: 0.3519
Epoch 291/2000
2/2 [==============================] - 0s 45ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 292/2000
2/2 [==============================] - 0s 43ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 293/2000
2/2 [==============================] - 0s 46ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 294/2000
2/2 [==============================] - 0s 44ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 295/2000
2/2 [==============================] - 0s 43ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 296/2000
2/2 [==============================] - 0s 41ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 297/2000
2/2 [==============================] - 0s 44ms/step - loss: 1.0978 - categorical_accuracy: 0.3519
Epoch 298/2000
2/2 [==============================] - 0s 42ms/step - loss: 1.0976 - categorical_accuracy: 0.351

2/2 [==============================] - 0s 43ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 363/2000
2/2 [==============================] - 0s 43ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 364/2000
2/2 [==============================] - 0s 47ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 365/2000
2/2 [==============================] - 0s 48ms/step - loss: 1.0979 - categorical_accuracy: 0.3519
Epoch 366/2000
2/2 [==============================] - 0s 46ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 367/2000
2/2 [==============================] - 0s 42ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 368/2000
2/2 [==============================] - 0s 45ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 369/2000
2/2 [==============================] - 0s 40ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 370/2000
2/2 [==============================] - 0s 42ms/step - loss: 1.0977 - categorical_accuracy: 0.351

2/2 [==============================] - 0s 42ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 435/2000
2/2 [==============================] - 0s 42ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 436/2000
2/2 [==============================] - 0s 43ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 437/2000
2/2 [==============================] - 0s 45ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 438/2000
2/2 [==============================] - 0s 46ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 439/2000
2/2 [==============================] - 0s 42ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 440/2000
2/2 [==============================] - 0s 43ms/step - loss: 1.0978 - categorical_accuracy: 0.3519
Epoch 441/2000
2/2 [==============================] - 0s 45ms/step - loss: 1.0978 - categorical_accuracy: 0.3519
Epoch 442/2000
2/2 [==============================] - 0s 43ms/step - loss: 1.0977 - categorical_accuracy: 0.351

2/2 [==============================] - 0s 33ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 507/2000
2/2 [==============================] - 0s 34ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 508/2000
2/2 [==============================] - 0s 33ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 509/2000
2/2 [==============================] - 0s 36ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 510/2000
2/2 [==============================] - 0s 37ms/step - loss: 1.0976 - categorical_accuracy: 0.3519
Epoch 511/2000
2/2 [==============================] - 0s 32ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 512/2000
2/2 [==============================] - 0s 34ms/step - loss: 1.0977 - categorical_accuracy: 0.3519
Epoch 513/2000
2/2 [==============================] - 0s 36ms/step - loss: 1.0978 - categorical_accuracy: 0.3519
Epoch 514/2000
2/2 [==============================] - 0s 34ms/step - loss: 1.0977 - categorical_accuracy: 0.351

KeyboardInterrupt: 

In [71]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 30, 64)            74240     
                                                                 
 lstm_4 (LSTM)               (None, 30, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_2 (Dense)             (None, 64)                4160      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 3)                 99        
                                                                 
Total params: 228803 (893.76 KB)
Trainable params: 228

In [72]:
res = model.predict(x_test)

1/1 [==============================] - 1s 536ms/step


In [74]:
actions[np.argmax(res[4])]

'Hello'

In [75]:
actions[np.argmax(y_test[4])]

'Sorry'

In [176]:
'''
def extract_coordinates(filepath):
    frameNr = 1
    cap = cv2.VideoCapture(filepath)
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
        while(frameNr < 31):
            ret, frame = cap.read()
            
            if not ret:
                break

            image, results = mediapipe_detection(frame, holistic)
            draw_styled_landmarks(image, results)
            cv2.imwrite(f"C:/Users/keith/OneDrive/Desktop/Videos/frame_{frameNr}.jpg", image)
            frameNr += 1
                    
    cap.release()
'''

s,s,s,